<a href="https://colab.research.google.com/github/shubham14yadav/Prototype-Dashboard-for-A-B-testing/blob/main/A_B_Testing_Prototype_Dashboard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#pip install dash pandas scipy

In [3]:
import base64
import io

import dash
from dash import dcc, html, Input, Output, State
import dash_table
import pandas as pd
from scipy import stats

app = dash.Dash(__name__)


app.layout = html.Div([
    html.H1("A/B Testing Dashboard"),
    dcc.Upload(
        id='upload-data',
        children=html.Div(['Drag and Drop or ', html.A('Select Files')]),
        style={
            'width': '100%',
            'height': '60px',
            'lineHeight': '60px',
            'borderWidth': '1px',
            'borderStyle': 'dashed',
            'borderRadius': '5px',
            'textAlign': 'center',
            'margin': '10px'
        },
        multiple=False
    ),
    html.Div(id='output-data-upload'),
    html.Div([
        html.Div([
            dcc.Dropdown(id='dropdown-a', placeholder='Select Group A'),
        ], style={'width': '48%', 'display': 'inline-block'}),
        html.Div([
            dcc.Dropdown(id='dropdown-b', placeholder='Select Group B'),
        ], style={'width': '48%', 'float': 'right', 'display': 'inline-block'})
    ]),
    html.Button('Run Test', id='run-test', n_clicks=0),
    html.Div(id='test-result')
])

# Parsing uploaded data
def parse_contents(contents, filename):
    content_type, content_string = contents.split(',')
    decoded = base64.b64decode(content_string)
    try:
        if 'csv' in filename:
            df = pd.read_csv(io.StringIO(decoded.decode('utf-8')))
        elif 'xls' in filename:
            df = pd.read_excel(io.BytesIO(decoded))
        else:
            return html.Div(['There was an error processing this file.'])
        return df
    except Exception as e:
        print(e)
        return html.Div(['There was an error processing this file.'])

# Uploading data
@app.callback(Output('output-data-upload', 'children'),
              Input('upload-data', 'contents'),
              State('upload-data', 'filename'))
def update_output(contents, filename):
    if contents is not None:
        df = parse_contents(contents, filename)
        return html.Div([
            dash_table.DataTable(
                data=df.to_dict('records'),
                columns=[{'name': i, 'id': i} for i in df.columns]
            ),
            html.Hr(),
            html.Div('Select the columns for A/B Testing:')
        ])
    else:
        return html.Div(['Upload a file to get started.'])

# Updating dropdown options
@app.callback(
    [Output('dropdown-a', 'options'),
     Output('dropdown-b', 'options')],
    Input('upload-data', 'contents'),
    State('upload-data', 'filename'))
def update_dropdown(contents, filename):
    if contents:
        df = parse_contents(contents, filename)
        return [[{'label': col, 'value': col} for col in df.columns]] * 2
    return [[], []]

# Run A/B test
@app.callback(
    Output('test-result', 'children'),
    Input('run-test', 'n_clicks'),
    [State('dropdown-a', 'value'),
     State('dropdown-b', 'value'),
     State('upload-data', 'contents'),
     State('upload-data', 'filename')])
def run_ab_test(n_clicks, group_a, group_b, contents, filename):
    if n_clicks > 0 and group_a and group_b and contents:
        df = parse_contents(contents, filename)
        t_stat, p_value = stats.ttest_ind(df[group_a].dropna(), df[group_b].dropna())
        return html.Div([
            html.H3('A/B Test Result'),
            html.P(f'T-statistic: {t_stat}'),
            html.P(f'P-value: {p_value}'),
            html.P('There is a significant difference between the two groups.' if p_value < 0.05 else 'There is no significant difference between the two groups.')
        ])
    return html.Div()


if __name__ == '__main__':
    app.run_server(debug=True)


<ipython-input-3-f555ba709981>:6: UserWarning: 
The dash_table package is deprecated. Please replace
`import dash_table` with `from dash import dash_table`

Also, if you're using any of the table format helpers (e.g. Group), replace 
`from dash_table.Format import Group` with 
`from dash.dash_table.Format import Group`
  import dash_table


<IPython.core.display.Javascript object>